In [ ]:
### Custom user input parameters:
# Would you like Eh (volts), pe-, logfO2, or Ah (kcal)?
output_type <- "Eh" # "Eh", "pe", "logfO2", or "Ah"

### End of custom user input parameters

In [ ]:
setwd("rxn_3o")

In [ ]:
n_files <- length(list.files())/3
sample_files <- paste(1:n_files, ".3o", sep="")

In [ ]:
master_df <- data.frame(sample = character(0), stringsAsFactors=FALSE)

for(this_file in sample_files){

  # read .3o file as a string
  fileName <- this_file
  extractme <- readChar(fileName, file.info(fileName)$size) # readChar requires filesize, which is obtained from file.info()$size
    
  # bind Eh to master data frame
  master_df[nrow(master_df)+1, ] <- rep(NA, ncol(master_df)) # create a new row
  master_df[nrow(master_df), "sample"] <- this_file
#   master_df[nrow(master_df), "Eh_NBS_pH_scale"] <- this_Eh


  # string to isolate the species saturation section:
  front_trim <- "^.*\n\n\n\n                --- Aqueous Redox Reactions ---\n\n   Couple                           Eh, volts      pe-      log fO2   Ah, kcal\n\n\\s+"

  # check if species block exists. If not, skip processing the file:
  if (grepl(front_trim, extractme) == FALSE){
    next # skip to next sample file
  }

  # trim off everything before:
  species_trailing <- sub(front_trim, "", extractme)
  # trim off everything after:
  species_block <- sub("\n\n.*$", "", species_trailing)
  # replace all commas with underscores
  species_block <- gsub(",", "_", species_block)
  # split into substrings, each representing a separate row in the table
  species_block <- strsplit(species_block, "\n")


  #create an empty data frame to store results
  df <- data.frame(couple = character(0),
                   Eh = numeric(0),
                   pe = numeric(0),
                   logfO2 = numeric(0),
                   Ah = numeric(0),
                   stringsAsFactors = FALSE)
  # convert into dataframe
  for(this_row in species_block[[1]]){

    # remove any leading spaces
    this_row <- sub("^\\s+", "", this_row)
    # trim off everything after species name to get this_species
    this_species <- sub("\\s+.*$", "", this_row)
    # to get row numeric values, trim off everything before it
    aq_numerics <- sub("^\\S+\\s+", "", this_row)
    # to get molality, trim off everything after its value in aq_numerics
    Eh <- sub("\\s+.*$", "", aq_numerics)
    # to get log molality, trim off everything before and after it
    pe_trailing <- sub("^\\S+\\s+", "", aq_numerics)
    pe <- sub("\\s+.*$", "", pe_trailing)
    # to get log gamma, trim off everything before and after it
    logfO2_trailing <- sub("^\\S+\\s+\\S+\\s+", "", aq_numerics)
    logfO2 <- sub("\\s+.*$", "", logfO2_trailing)
    # to get log activity, trim off everything before and after it
    Ah_trailing <- sub("^\\S+\\s+\\S+\\s+\\S+\\s+", "", aq_numerics)
    Ah <- sub("\\s+.*$", "", Ah_trailing)

    # create a dataframe with results
    this_df <- data.frame(couple = this_species,
                          Eh = as.numeric(Eh),
                          pe = as.numeric(pe),
                          logfO2 = as.numeric(logfO2),
                          Ah = as.numeric(Ah),                   
                          stringsAsFactors = FALSE)
    # bind results to dataframe
    df <- rbind(df, this_df)
  }
  df

  # add species affinities to master_df
  for(this_species in df[, "couple"]){
    # if this_species does not have a column in the master_df, add it
    if (!(this_species %in% colnames(master_df))){
      master_df <- cbind(master_df, rep(NA, nrow(master_df))) # add extra column
      colnames(master_df)[ncol(master_df)] <- this_species # rename added column
    }

    master_df[nrow(master_df), this_species] <- df[which(df[, "couple"] == this_species), output_type]
  }
}

# sort species columns in alphabetical order
unsorted_species_names <- colnames(master_df[, 2:ncol(master_df)])
master_df <- master_df[, c("sample", sort(unsorted_species_names))]
master_df

setwd("../")
write.csv(master_df, paste("redox_", output_type,".csv", sep=""))